In [ ]:
import os
import re
import time
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

torch.manual_seed(1)

In [4]:
import preprocess
from net.model import FesNet

stiffness, mass = preprocess.task()

In [8]:
print(type(stiffness))
print(len(mass[0]))

<class 'tuple'>
153


In [9]:
import utils.load_acc

